# ETL Pipeline Preparation
Follow the instructions below to help you create your ETL pipeline.
### 1. Import libraries and load datasets.
- Import Python libraries
- Load `messages.csv` into a dataframe and inspect the first few lines.
- Load `categories.csv` into a dataframe and inspect the first few lines.

In [435]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

In [436]:
# load messages dataset
messages = pd.read_csv("disaster_messages.csv")
messages.head()

,id,message,original,genre
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct


"original" is the original non-English language

In [437]:
messages.isna().mean()

id          0.000000
message     0.000000
original    0.612009
genre       0.000000
dtype: float64

"original" language is blank when message is in English
Other fields don't have empty values
So 'original' language can be dropped

In [438]:
messages.drop(columns = "original", inplace=True)
messages.head(5)

,id,message,genre
0,2,Weather update - a cold front from Cuba that c...,direct
1,7,Is the Hurricane over or is it not over,direct
2,8,Looking for someone but no name,direct
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,direct
4,12,"says: west side of Haiti, rest of the country ...",direct


In [439]:
messages["genre"].value_counts()

news      13068
direct    10782
social     2398
Name: genre, dtype: int64

Messages through social media is relatively low

In [440]:
# load categories dataset
categories = pd.read_csv("disaster_categories.csv")
categories.head()

,id,categories
0,2,related-1;request-0;offer-0;aid_related-0;medi...
1,7,related-1;request-0;offer-0;aid_related-1;medi...
2,8,related-1;request-0;offer-0;aid_related-0;medi...
3,9,related-1;request-1;offer-0;aid_related-1;medi...
4,12,related-1;request-0;offer-0;aid_related-0;medi...


Just two fields in the CSV file, but categories are concatenated in the 'categories' columns

In [441]:
messages.isna().mean()

id         0.0
message    0.0
genre      0.0
dtype: float64

In [442]:
categories.isna().mean()

id            0.0
categories    0.0
dtype: float64

No blank message and category columns

In [443]:
print (messages.shape)
print (categories.shape)

(26248, 3)
(26248, 2)


Messages and categories have the same number of records

### 2. Merge datasets.
- Merge the messages and categories datasets using the common id
- Assign this combined dataset to `df`, which will be cleaned in the following steps

In [444]:
# merge datasets
df = pd.merge(messages, categories, on="id", how="inner")

print (messages.shape)
print (categories.shape)
print (df.shape)
df.isna().sum()

(26248, 3)
(26248, 2)
(26386, 4)


id            0
message       0
genre         0
categories    0
dtype: int64

df has more records which implies that there is an ID mismatch or there are duplicate ID's

In [445]:
messages.drop_duplicates(subset="id", inplace=True)
categories.drop_duplicates(subset="id", inplace=True)
df = pd.merge(messages, categories, on="id", how="inner")

print (messages.shape)
print (categories.shape)
print (df.shape)

(26180, 3)
(26180, 2)
(26180, 4)


number of records in df is consistent with the recods in messages and categories

In [446]:
df.isna().mean()

id            0.0
message       0.0
genre         0.0
categories    0.0
dtype: float64

### 3. Split `categories` into separate category columns.
- Split the values in the `categories` column on the `;` character so that each value becomes a separate column. You'll find [this method](https://pandas.pydata.org/pandas-docs/version/0.23/generated/pandas.Series.str.split.html) very helpful! Make sure to set `expand=True`.
- Use the first row of categories dataframe to create column names for the categories data.
- Rename columns of `categories` with new column names.

In [447]:
# create a dataframe of the 36 individual category columns
categories = categories['categories'].str.split(';', expand=True)
categories.shape

(26180, 36)

In [448]:
# select the first row of the categories dataframe
firstrow = categories.head(1).values[0]
firstrow

array(['related-1', 'request-0', 'offer-0', 'aid_related-0',
       'medical_help-0', 'medical_products-0', 'search_and_rescue-0',
       'security-0', 'military-0', 'child_alone-0', 'water-0', 'food-0',
       'shelter-0', 'clothing-0', 'money-0', 'missing_people-0',
       'refugees-0', 'death-0', 'other_aid-0', 'infrastructure_related-0',
       'transport-0', 'buildings-0', 'electricity-0', 'tools-0',
       'hospitals-0', 'shops-0', 'aid_centers-0',
       'other_infrastructure-0', 'weather_related-0', 'floods-0',
       'storm-0', 'fire-0', 'earthquake-0', 'cold-0', 'other_weather-0',
       'direct_report-0'], dtype=object)

In [449]:
# use this row to extract a list of new column names for categories.
# one way is to apply a lambda function that takes everything 
# up to the second to last character of each string with slicing
def strip_last(s):
    return s[:-2]

category_colnames = np.array(list(map(strip_last, firstrow)))
print(category_colnames)

['related' 'request' 'offer' 'aid_related' 'medical_help'
 'medical_products' 'search_and_rescue' 'security' 'military'
 'child_alone' 'water' 'food' 'shelter' 'clothing' 'money'
 'missing_people' 'refugees' 'death' 'other_aid' 'infrastructure_related'
 'transport' 'buildings' 'electricity' 'tools' 'hospitals' 'shops'
 'aid_centers' 'other_infrastructure' 'weather_related' 'floods' 'storm'
 'fire' 'earthquake' 'cold' 'other_weather' 'direct_report']


In [450]:
# rename the columns of `categories`
categories.columns = category_colnames
categories.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
1,related-1,request-0,offer-0,aid_related-1,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-1,floods-0,storm-1,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
2,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
3,related-1,request-1,offer-0,aid_related-1,medical_help-0,medical_products-1,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
4,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0


### 4. Convert category values to just numbers 0 or 1.
- Iterate through the category columns in df to keep only the last character of each string (the 1 or 0). For example, `related-0` becomes `0`, `related-1` becomes `1`. Convert the string to a numeric value.
- You can perform [normal string actions on Pandas Series](https://pandas.pydata.org/pandas-docs/stable/text.html#indexing-with-str), like indexing, by including `.str` after the Series. You may need to first convert the Series to be of type string, which you can do with `astype(str)`.

In [451]:
categories['related'] = categories['related'].astype('str').str.replace('2', '1')

In [452]:
for column in categories:
    # set each value to be the last character of the string
    categories[column] = categories[column].str[-1]
    
    # convert column from string to numeric
    categories[column] = categories[column].astype(int)
categories.shape

(26180, 36)

In [453]:
categories.apply(pd.Series.value_counts)

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,6116,21716,26062,15339,24099,24869,25456,25709,25321,26180.0,...,25871,25029,18894,24031,23740,25898,23728,25652,24804,21116
1,20064,4464,118,10841,2081,1311,724,471,859,NaN,...,309,1151,7286,2149,2440,282,2452,528,1376,5064


### 5. Replace `categories` column in `df` with new category columns.
- Drop the categories column from the df dataframe since it is no longer needed.
- Concatenate df and categories data frames.

In [454]:
# drop the original categories column from `df`
df.drop(columns="categories", inplace=True)

df.shape

(26180, 3)

In [455]:
categories

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26243,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26244,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26245,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26246,1,0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [456]:
# concatenate the original dataframe with the new `categories` dataframe
df = pd.concat([df, categories], axis=1)
df = df.dropna()
df = df.convert_dtypes()
df

,id,message,genre,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,direct,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,direct,1,0,0,1,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,direct,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",direct,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26175,30261,The training demonstrated how to enhance micro...,news,1,0,0,1,1,1,0,...,0,1,1,1,1,1,1,0,1,0
26176,30262,A suitable candidate has been selected and OCH...,news,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26177,30263,"Proshika, operating in Cox's Bazar municipalit...",news,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26178,30264,"Some 2,000 women protesting against the conduc...",news,1,0,0,1,0,0,0,...,0,0,1,1,0,0,0,0,0,0


### 6. Remove duplicates.
- Check how many duplicates are in this dataset.
- Drop the duplicates.
- Confirm duplicates were removed.

In [457]:
# check number of duplicates
df.duplicated().sum()

0

In [458]:
# drop duplicates
df = df.drop_duplicates()

In [459]:
# check number of duplicates
df.duplicated().sum()

0

### 7. Save the clean dataset into an sqlite database.
You can do this with pandas [`to_sql` method](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.to_sql.html) combined with the SQLAlchemy library. Remember to import SQLAlchemy's `create_engine` in the first cell of this notebook to use it below.

In [460]:
engine = create_engine('sqlite:///DisasterResponse.db')
df.to_sql('InsertTableName', engine, index=False, if_exists="replace")

### 8. Use this notebook to complete `etl_pipeline.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database based on new datasets specified by the user. Alternatively, you can complete `etl_pipeline.py` in the classroom on the `Project Workspace IDE` coming later.